This GBDX workflow takes an 8-bit pansharpened image, jpeg-compresses it, and uploads it to Mapbox.
jpeg-compression reduces the size of the image drastically without a visible effect on quality.
More info in this great little blog post: http://blog.cleverelephant.ca/2015/02/geotiff-compression-for-dummies.html

In [ ]:
# create a gbdxtools interface

import os
os.environ['GBDX_USERNAME'] = 
os.environ['GBDX_PASSWORD'] = 
os.environ['GBDX_CLIENT_ID'] =  
os.environ['GBDX_CLIENT_SECRET'] = 

import gbdxtools

gbdx = gbdxtools.Interface()

In [ ]:
# use gdal-cli to jpeg compress
compress = gbdx.Task('gdal-cli')

# the input image is a pansharpened strip over Adelaide (catalog id 1040010014800C00)
compress.inputs.data = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/swimming-pools/images'

compress.inputs.execution_strategy = 'runonce'
compress.inputs.command = 'gdal_translate  $indir/*.tif $outdir/compressed.tif -b 1 -b 2 -b 3 -co COMPRESS=JPEG -co PHOTOMETRIC=YCBCR -co TILED=YES'

# use upload-to-mapbox to create tilestack
utom = gbdx.Task('upload-to-mapbox')
utom.inputs.input = compress.outputs.data.value

# specify Mapbox tileset name
utom.inputs.tileset_name = 'swimming-pools-adelaide'

# specify access token to upload
utom.inputs.token = 'ENTER VALID TOKEN HERE'

# specify timeout in seconds
utom.inputs.timeout = '1200'

wf = gbdx.Workflow([compress, utom])
wf.execute()

In [ ]:
wf.status

You can preview the slippy map using the following url. The map id is platformstories.swimming-pools-adelaide.

In [ ]:
from IPython.display import IFrame
IFrame('https://api.mapbox.com/v4/platformstories.swimming-pools-adelaide/page.html?access_token=pk.eyJ1IjoicGxhdGZvcm1zdG9yaWVzIiwiYSI6ImNpeTZkeWlvOTAwNm0yeHFocHFyaGFleDcifQ.wOsbVsS0NXKrWeX2bQwc-g', width=1600, height=800)

An example of how to reference a tilestack using Mapbox GL is found [here](http://gbdxstories.digitalglobe.com/pages/osm-lulc/cities.html).